<a href="https://colab.research.google.com/github/MarisabelC/QA_Word2Vec/blob/main/AnswerEngine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download Model

In [ ]:
import os

%cd /content
if not os.path.exists("GoogleNews-vectors-negative300.bin.gz"):
  !wget -P /content/ -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"
else:
  print("word2vec already downloaded")

/content
--2020-09-18 22:41:50--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.42.254
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.42.254|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘/content/GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  44.6MB/s    in 36s     

2020-09-18 22:42:26 (43.7 MB/s) - ‘/content/GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



In [ ]:
from gensim.models import KeyedVectors
import gensim.downloader as api

# EMBEDDING_FILE = 'GoogleNews-vectors-negative300.bin.gz'
# word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)
word2vec = api.load('word2vec-google-news-300')

[=================================================-] 99.4% 1652.9/1662.8MB downloaded


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


# SpacyMatching.py


In [ ]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
  
def remove_stopword(sentence):
  
  stop_words = set(stopwords.words('english')) 
    
  word_tokens = word_tokenize(sentense) 
    
  filtered_sentence = [w for w in word_tokens if not w in stop_words] 
    
  return filtered_sentence

In [ ]:
import scipy
import numpy as np
from scipy import spatial

index2word_set = set(word2vec.wv.index2word)

def avg_feature_vector(sentence, model, num_features, index2word_set):
    # stop_words = ['a','an','the']
    # words = sentence.split()
    feature_vec = np.zeros((num_features, ), dtype='float32')
    n_words = 0
    doc = nlp(sentence)
    words = [token.text for token in doc if not token.is_stop]
    for word in words:
      # if word in index2word_set and not word in stop_words:
      if word in index2word_set:
        n_words += 1
        feature_vec = np.add(feature_vec, model[word])
    if (n_words > 0):
        feature_vec = np.divide(feature_vec, n_words)
    return feature_vec    

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """


In [ ]:
spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS
print('Number of stop words: %d' % len(spacy_stopwords))
print('First ten stop words: %s' % list(spacy_stopwords)[:10])

In [ ]:
def matching(a, b):
    index2word_set = set(word2vec.wv.index2word)
    s1_afv = avg_feature_vector(a, model=word2vec, num_features=300, index2word_set=index2word_set)
    s2_afv = avg_feature_vector(b, model=word2vec, num_features=300, index2word_set=index2word_set)
    sim = 1 - spatial.distance.cosine(s1_afv, s2_afv)
    return sim

# QuestionAnswerEngineTask.py 

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
import spacy
import pandas as pd
import gensim.downloader as api
import re

lemmatizer = WordNetLemmatizer()
# nltk.download('punkt')
# pd.set_option('display.max_colwidth', 200)
nlp = spacy.load("en_core_web_sm")



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
import math

def returnresult(text, question, numofanswers):
    # sentence_list = re.split('\.”|\.',text)
    # sentence_list = [sentence.strip() for sentence in sentence_list]
    
    # for sentence in sentence_list:
    #   print(sentence)
    # question = question
    # text = text

    # questiondoc = nlp(question)
   
    doc = nlp(text)

    sentences = [sent.string.strip() for sent in doc.sents if len(sent) > 2]
    # for sentence in sentences:
    #   print(sentence)
    
    scores ={}
    for sentence in sentences:
      
      scores[sentence] = matching(question,sentence)
      # scores[sentence] = word2vec.wmdistance(question, sentence)

    sort_scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    print(round(sort_scores[0][1],2))
    # if round(sort_scores[0][1],2) <= 0.5 or math.isnan(sort_scores[0][1]) :
    if math.isnan(sort_scores[0][1]) :
        return scores , ['there was no match']
    else:
        # debugging_list = []
        result_list = []
        
        for i in range(len(sort_scores)):
          # sentence_index = sentences.index(sort_scores[i][0])
          result_list.append(sort_scores[i][0])
          # result_list.append(matching_sentence(sentence_list, sort_scores[i][0], sentence_index))
        
        # print(result_list)
        print('1st: ', sort_scores[0])
        print('2nd: ', sort_scores[1])
        print('3rd: ', sort_scores[2])
    
    return scores, result_list

    


# test_cases_engine.py 


In [ ]:
import json

question_list_file_path = "question_list.json"

with open(question_list_file_path, 'r',  encoding="utf8") as file:

    data = file.read()

question_list = json.loads(data)

correctAnswer = 0
top1= 0

for q in question_list:
    print("************************************************************")
    print("Testing ", q['name'])

    text_file_path =  q['file']
    with open(text_file_path, 'r', encoding="utf8") as file:
        text_data = file.read()

    scores, res = returnresult(text_data,  q['question'], 3)
    print(q['question'])
  

    # provided_answer=  remove_abbrev(q['Answer'].lower()) 

    # print('answer:')
    for i, answer in enumerate (res):
        correct = False 

        # print(answer)
        if  answer.lower() == q['Answer'].lower():
            if i == 0:
              top1 += 1
            correctAnswer+= 1
            correct = True
            print('correct answer')
            break
    if correct == False and q['Answer'] in scores:
      print('\n incorrect:' , scores[q['Answer']])
    print('\ncorrect answer: ' ,q['Answer'] )    
    print(res)
    print()

print('accuracy: ' , correctAnswer / len(question_list), ' correct answer: ' , correctAnswer, 'first option:' , top1)

************************************************************
Testing  t1-q1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  
/usr/local/lib/python3.6/dist-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in float_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


nan
What is my name?

correct answer:  my name is nathan ji.
['there was no match']

************************************************************
Testing  t1-q2
0.77
1st:  ('I am 14 years old.', 0.771997332572937)
2nd:  ('My name is Nathan Ji.', 0.21168991923332214)
3rd:  ('I go to Portola High school.', 0.10335738211870193)
How old am I?
correct answer

correct answer:  i am 14 years old.
['I am 14 years old.', 'My name is Nathan Ji.', 'I go to Portola High school.']

************************************************************
Testing  t1-q3
0.64
1st:  ('I go to Portola High school.', 0.6365323662757874)
2nd:  ('I am 14 years old.', 0.19540956616401672)
3rd:  ('My name is Nathan Ji.', 0.08180298656225204)
Where do I go to school?
correct answer

correct answer:  i go to portola high school.
['I go to Portola High school.', 'I am 14 years old.', 'My name is Nathan Ji.']

************************************************************
Testing  t2-q1
nan
What is the name of the 6th preside

# Test a single question

In [ ]:
# q = question_list[24]
# print("************************************************************")
# print("Testing ", q['name'])

# text_file_path =  q['file']
# with open(text_file_path, 'r', encoding="utf8") as file:
#     text_data = file.read()

# scores, res = returnresult(text_data,  q['question'], 3)
# print(q['question'])


# # provided_answer=  remove_abbrev(q['Answer'].lower()) 

# # print('answer:')
# for i, answer in enumerate (res):
#     correct = False 

#     # print(answer)
#     if  answer.lower() == q['Answer'].lower():
#         if i == 0:
#           top1 += 1
#         correctAnswer+= 1
#         correct = True
#         print('correct answer')
#         break
# if correct == False and q['Answer'] in scores:
#   print('\n incorrect:' , scores[q['Answer']])
# print('\ncorrect answer: ' ,q['Answer'] )    
# print(res)
# print()

# Old code to remove abbreviation


In [ ]:
# def remove_dot_acronym(s):
#   m=re.search('(.*?)(([a-zA-Z]\.){2,})(.*)', s)
#   if m:
#       replacement=''.join(m.group(2).split('.'))
#       s=m.group(1)+replacement+m.group(4)
#       return s
#   else: 
#     return m

# def remove_abbrev(text):
#   temp = remove_dot_acronym(text)
#   while temp != None:
#     text= temp
#     temp = remove_dot_acronym(text)
    
#   text = re.sub(r"(?<= [.(a-zA-Z)]{3})\.(?!=(\n))", '', text)
#   text = re.sub(r"(?<= [a-zA-Z]{2})\.(?!=(\n))", '', text)
#   text = re.sub(r"(?<= [a-zA-Z]{1})\.(?!=(\n))", '', text)
  
#   return text

# def matching_sentence(sentence_list, sentence, index_sentence):
#     possible_sentence=''
    
#     # print(sentence_list)
#     current_pos = 0
#     temp_pos = 0
#     for i in range (index_sentence, len(sentence_list)):
#       # print(sentence_list[i])
#       # print(sentence)
#       for j in range (len(sentence_list[i])):
#         # print(sentence[temp_pos],' - ', sentence_list[i][j].lower())
#         if temp_pos < len(sentence) and sentence[temp_pos] == sentence_list[i][j].lower() :
#             # print(sentence[temp_pos],' - ', sentence_list[i][j].lower())
#             temp_pos += 1
#         else:
#           temp_pos = current_pos  
#           break

#       if temp_pos != current_pos :
#           possible_sentence += sentence_list[i] + '. '
#           if temp_pos > 10:
#             for k in range (i+1, len(sentence_list)):
#               if len(possible_sentence) >= len(sentence):
#                 break
#               possible_sentence += sentence_list[k] + '. '
#               break
          
#           else:
#             temp_pos+=1
#             current_pos = temp_pos  
        
#     return possible_sentence


In [ ]:

# print(remove_dot_acronym('.c.'))

# print(remove_abbrev('l.g.b.t.q.'))